# Introduction

In today's competitive hospitality industry, understanding and improving guest satisfaction is critical for hosts, property managers, and platform administrators to maintain high occupancy rates and positive reviews.

Airbnb, as one of the leading platforms in the short-term rental market, provides a wealth of data through guest reviews. This project will implemented RoBERTa from HuggingFace’s Transformers library for sentiment analysis from review data.

The primary objective of this project is to develop a sentiment analysis pipeline capable of measuring guest satisfaction over time by analyzing sentiment from Airbnb reviews. This system will enable hosts and property managers to track changes in guest satisfaction, evaluate the effectiveness of improvements or changes made to their properties or services, and make data-driven decision-making to enhance guest experiences.

# Dataset

This comprehensive [dataset](https://data.insideairbnb.com/south-africa/wc/cape-town/2024-12-28/data/reviews.csv.gz) from [Inside Airbnb](http://insideairbnb.com/), comprising over `500,000 rows`, offers a broad perspective on guest experiences in Cape Town, South Africa. The dataset snapshot used for this analysis was collected on December 28, 2024.

The dataset comprises several key features that are crucial for understanding guest sentiment and experiences:

- **listing_id:** A unique identifier for each Airbnb listing.
- **reviewer_name:** The name of the guest who wrote the review.
- **comments:** The text content of the guest's review.
- **date:** The date when the review was submitted.



# Setup

## 1. Load Essential Libraries

In [1]:
# Import neccessary libraries
import os
import re
from tqdm import tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

%matplotlib inline

## 2. Dataset

In [2]:
import gzip
import shutil
import os
import requests

# Create the 'data' directory if it doesn't exist
os.makedirs('data', exist_ok=True)

# Download the gzipped file
request = requests.get("https://data.insideairbnb.com/south-africa/wc/cape-town/2024-12-28/data/reviews.csv.gz", stream=True)

# Save and decompress simultaneously
with open("data/reviews.csv.gz", "wb") as file:
    shutil.copyfileobj(request.raw, file)

with gzip.open("data/reviews.csv.gz", "rt", encoding="utf-8") as gz_file:
    with open("data/reviews.csv", "wt", encoding="utf-8") as outfile:
        shutil.copyfileobj(gz_file, outfile)

# Exploratory Data Analysis (EDA) and Data Cleaning

### 1. Data Loading and Insepection

In [3]:
df = pd.read_csv('data/reviews.csv')  # Data loading
df.head()  # Display the first few rows

,listing_id,id,date,reviewer_id,reviewer_name,comments
0,3191,4888238,2013-05-31,5737473,Kathleen,Great home away from home! Bridgette and Marth...
1,3191,9128602,2013-12-09,8170322,Anita,Das Cottage liegt ruhig und sicher. Wir haben...
2,3191,9924130,2014-01-20,4039279,Zacki,This cottage was a great base from which to ex...
3,3191,16659537,2014-07-31,9729939,Doug,I had a great stay. All my needs were well ex...
4,3191,23247470,2014-11-26,9681619,Christopher,Excellent host. She provided everything we cou...


In [4]:
# Get data types and missing values
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 596775 entries, 0 to 596774
Data columns (total 6 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   listing_id     596775 non-null  int64 
 1   id             596775 non-null  int64 
 2   date           596775 non-null  object
 3   reviewer_id    596775 non-null  int64 
 4   reviewer_name  596774 non-null  object
 5   comments       596654 non-null  object
dtypes: int64(3), object(3)
memory usage: 27.3+ MB


## 2. Handling Missing Values

In [5]:
# Check for missing values
print(df.isnull().sum())

listing_id         0
id                 0
date               0
reviewer_id        0
reviewer_name      1
comments         121
dtype: int64


In [6]:
# Drop rows with missing values in 'reviewer_name' and 'comments'
df.dropna(subset=['reviewer_name', 'comments'], inplace=True)

# Reset index after dropping rows
df.reset_index(drop=True, inplace=True)

## 3. Exploring Data Distributions

Identify popular listings or potential outliers.

In [7]:
listing_counts = df['listing_id'].value_counts().reset_index()
listing_counts.columns = ['Listing ID', 'Number of Reviews']
top_20_listings = listing_counts.head(20)  # Select the top 20 listings

# Sort in ascending order
top_20_listings = top_20_listings.sort_values(by='Number of Reviews', ascending=True)

fig = px.bar(
       top_20_listings,
       x='Number of Reviews',
       y='Listing ID',
       orientation='h',  # Horizontal bar chart
       title='Top 20 Most Reviewed Listings'
   )

# Update y-axis formatting
fig.update_yaxes(type='category') # Force Plotly to treat Listing ID as categorical

fig.show()

Distribution of reviews over time

In [8]:
review_counts = df.groupby('date')['listing_id'].count().reset_index()
review_counts.columns = ['Date', 'Number of Reviews']  # Rename columns

fig = px.line(
    review_counts,
    x='Date',
    y='Number of Reviews',
    title='Distribution of Reviews Over Time'
)

fig.show()

In [12]:
# Analyze for seasonality (example using monthly averages)

# Convert the 'date' column to datetime objects
df['date'] = pd.to_datetime(df['date'])

df['month'] = df['date'].dt.month

monthly_reviews = df.groupby('month')['listing_id'].count().reset_index()
monthly_reviews.columns = ['Month', 'Average Number of Reviews']

fig = px.bar(
    monthly_reviews,
    x='Month',
    y='Average Number of Reviews',
    title='Seasonality of Reviews (2010-2024)'
)

fig.show()

Analyze the length of comments

In [13]:
df['comment_length'] = df['comments'].str.len()

# Create the histogram
fig = px.histogram(df, x="comment_length", title="Distribution of Comment Lengths")
fig.update_xaxes(title_text="Comment Length (Number of Characters)")
fig.update_yaxes(title_text="Frequency")
fig.show()

# Preprocessing a Random Listing with Sufficient Reviews for Analysis

In [14]:
import random

# Minimum number of reviews required
min_reviews = 20  # Adjust as needed

# Select a random listing_id
while True:
    random_listing_id = random.choice(df['listing_id'].unique())
    filtered_reviews = df[df['listing_id'] == random_listing_id]

    # Check if the listing has enough reviews
    if len(filtered_reviews) >= min_reviews:
        break  # Exit the loop if enough reviews are found

print(f"Selected listing_id: {random_listing_id}")
print(filtered_reviews['listing_id'].value_counts())

Selected listing_id: 27022165
listing_id
27022165    29
Name: count, dtype: int64


## 1. Language Detection and Filtering Non-English Reviews

Detect and Remove Non-English Reviews

In [15]:
!pip install langdetect

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 12.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993223 sha256=5f8256f661aa35713d8c0ed6d3a6e3a94fc4b78dd514b27d1c9fccf271e2576f
  Stored in directory: /root/.cache/pip/wheels/0a/f2/b2/e5ca405801e05eb7c8ed5b3b4bcf1fcabcd6272c167640072e
Successfully built langdetect


In [16]:
from langdetect import detect, DetectorFactory
from langdetect.lang_detect_exception import LangDetectException

# Set seed for reproducibility in language detection: Ensuring Consistent Results
DetectorFactory.seed = 0

def detect_language(text):
    try:
        return detect(text)
    except LangDetectException:
        # If language detection fails, assume it's not English
        return 'unknown'

Apply Language Detection

In [17]:
# Apply Language Detection
filtered_reviews.loc[:, 'language'] = filtered_reviews['comments'].apply(detect_language)

# Display the language distribution
print(filtered_reviews['language'].value_counts())

# Filter Out Non-English Reviews
df_english = filtered_reviews[filtered_reviews['language'] == 'en']

# Reset index after filtering
df_english.reset_index(drop=True, inplace=True)

# Check the shape of the filtered dataset
print("Original dataset size:", filtered_reviews.shape)
print("Filtered dataset size:", df_english.shape)

language
en    28
tl     1
Name: count, dtype: int64
Original dataset size: (29, 9)
Filtered dataset size: (28, 9)


<ipython-input-17-e222c5c05587>:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



## 2. Clean Reviews

Clean review text by removing unnecessary characters, HTML tags, URLs, and special symbols.

In [18]:
def clean_text(text):
    text = re.sub(r'<.*?>', '', text)  #Remove HTML tags as they might confuse the model
    text = re.sub(r'http\S+|www.\S+', '', text)   # Remove URLs
    return text

df_english.loc[:, 'cleaned_comments'] = df_english['comments'].apply(clean_text)

print(df_english[['comments', 'cleaned_comments']].sample(5))

                                             comments  \
17  Wonderful host , very easy going and made me f...   
6   Vannesa is such a great host. I had the the pr...   
16                                       thanks a lot   
0   Vanessa is such a wonderful host, I am definit...   
19  I have went around a few Air BNB where the ser...   

                                     cleaned_comments  
17  Wonderful host , very easy going and made me f...  
6   Vannesa is such a great host. I had the the pr...  
16                                       thanks a lot  
0   Vanessa is such a wonderful host, I am definit...  
19  I have went around a few Air BNB where the ser...  


<ipython-input-18-fc3c187917fa>:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



# RoBERTa Model

In [20]:
# Import neccessary libraries

from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from scipy.special import softmax

In [21]:
MODEL = f"cardiffnlp/twitter-roberta-base-sentiment"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
model = AutoModelForSequenceClassification.from_pretrained(MODEL)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning:


The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.



config.json:   0%|          | 0.00/747 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


pytorch_model.bin:   0%|          | 0.00/499M [00:00<?, ?B/s]

In [22]:
example = df_english['cleaned_comments'].iloc[2]
print(example)

Nice accommodation run by really friendly and helpful host.  Made me feel right at home :-)


In [23]:
encoded_text = tokenizer(example, return_tensors='pt', truncation=True, padding=True, max_length=512)
output = model(**encoded_text)
scores = output[0][0].detach().numpy()
scores = softmax(scores)
scores_dict = {
    'roberta_neg' : scores[0],
    'roberta_neu' : scores[1],
    'roberta_pos' : scores[2]
}
print(scores_dict)

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


{'roberta_neg': np.float32(0.0014291845), 'roberta_neu': np.float32(0.01206059), 'roberta_pos': np.float32(0.9865102)}


In [24]:
from tqdm import tqdm

def polarity_scores_roberta(example):
  encoded_text = tokenizer(example, return_tensors='pt', truncation=True, padding=True, max_length=512)
  output = model(**encoded_text)
  scores = output[0][0].detach().numpy()
  scores = softmax(scores)
  scores_dict = {
    'roberta_neg' : scores[0],
    'roberta_neu' : scores[1],
    'roberta_pos' : scores[2]
  }
  return scores_dict

In [25]:
results = {}
for i, row in tqdm(df_english.iterrows(), total=len(df_english)):
  try:
    text = row['cleaned_comments']
    myid = row['id']

    # RoBERTa Sentiment Analysis
    roberta_result = polarity_scores_roberta(text)

    # Storing Results
    results[myid] = roberta_result
  except RuntimeError:
    print(f'Broke for id {myid}')

 18%|█▊        | 5/28 [00:00<00:04,  5.68it/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

100%|██████████| 28/28 [00:10<00:00,  2.72it/s]


In [26]:
results_df = pd.DataFrame(results).T
results_df = results_df.reset_index().rename(columns={'index': 'id'})
results_df = filtered_reviews.merge(results_df, how='left')

In [27]:
results_df.head()

,listing_id,id,date,reviewer_id,reviewer_name,comments,month,comment_length,language,roberta_neg,roberta_neu,roberta_pos
0,27022165,402641003,2019-01-19,225411582,Naledi,"Vanessa is such a wonderful host, I am definit...",1,223,en,0.001638,0.005513,0.992848
1,27022165,406043659,2019-01-28,237968668,Tabitha,I would never have forgiven myself if I would ...,1,230,en,0.005183,0.020476,0.974340
2,27022165,430825859,2019-03-31,34833404,Tony,Nice accommodation run by really friendly and ...,3,91,en,0.001429,0.012061,0.986510
3,27022165,440028241,2019-04-20,34833404,Tony,This is the second time that I have stayed wit...,4,95,en,0.001859,0.018720,0.979421
4,27022165,446798479,2019-05-02,46204540,Heather,"Vanessa, thanks for making my son and his girl...",5,129,en,0.001064,0.008573,0.990362


In [28]:
sentiment_distribution = results_df[['roberta_neg', 'roberta_neu', 'roberta_pos']].mean()
print(sentiment_distribution)

roberta_neg    0.002218
roberta_neu    0.022453
roberta_pos    0.975329
dtype: float32


# Results

## 1. Visualize guest satisfaction trends

### Year Averages

In [30]:
# Group by year and calculate the mean sentiment scores
yearly_sentiment = results_df.groupby(results_df['date'].dt.year)[['roberta_neg', 'roberta_neu', 'roberta_pos']].mean().reset_index()

# Rename columns for better readability in the plot
yearly_sentiment.columns = ['Year', 'Negative', 'Neutral', 'Positive']

# Create the Plotly Express line chart
fig = px.line(
    yearly_sentiment,
    x='Year',
    y=['Negative', 'Neutral', 'Positive'],
    title='Guest Satisfaction Trends Over Time (Yearly Averages)',
    labels={'value': 'Average Sentiment Score'},  # Label for y-axis
    markers=True  # Add markers to data points
)

# Customize the layout
fig.update_layout(
    xaxis_title='Year',
    yaxis_title='Average Sentiment Score',
    legend_title_text='Sentiment',
    xaxis=dict(tickmode='linear', showgrid=True, gridcolor='lightgray', gridwidth=0.5),  # Show all years on x-axis, add grid properties
    yaxis=dict(showgrid=True, gridcolor='lightgray', gridwidth=0.5), # Add grid properties to y-axis
)

# Show the plot
fig.show()

### Quaterly Averagas

In [32]:
# Group by quarter and calculate the mean sentiment scores
quarterly_sentiment = results_df.groupby(results_df['date'].dt.to_period('Q'))[['roberta_neg', 'roberta_neu', 'roberta_pos']].mean().reset_index()

# Convert the PeriodIndex to strings for plotting
quarterly_sentiment['Quarter'] = quarterly_sentiment['date'].astype(str)

# Create the Plotly Express line chart
fig = px.line(
    quarterly_sentiment,
    x='Quarter',
    y=['roberta_neg', 'roberta_neu', 'roberta_pos'],
    title='Guest Satisfaction Trends Over Time (Quarterly Averages)',
    labels={'value': 'Average Sentiment Score'},
    markers=True
)

# Customize the layout
fig.update_layout(
    xaxis_title='Quarter',
    yaxis_title='Average Sentiment Score',
    legend_title_text='Sentiment',
    xaxis=dict(tickmode='linear', showgrid=True, gridcolor='lightgray', gridwidth=0.5),  # Show all years on x-axis, add grid properties
    yaxis=dict(showgrid=True, gridcolor='lightgray', gridwidth=0.5), # Add grid properties to y-axis
)

# Show the plot
fig.show()

### Monthly Averages

In [34]:
# Group by month and calculate the mean sentiment scores
monthly_sentiment = results_df.groupby(results_df['date'].dt.to_period('M'))[['roberta_neg', 'roberta_neu', 'roberta_pos']].mean().reset_index()

# Convert the PeriodIndex to strings for plotting
monthly_sentiment['Month'] = monthly_sentiment['date'].astype(str)

# Create the Plotly Express line chart
fig = px.line(
    monthly_sentiment,
    x='Month',
    y=['roberta_neg', 'roberta_neu', 'roberta_pos'],
    title='Guest Satisfaction Trends Over Time (Monthly Averages)',
    labels={'value': 'Average Sentiment Score'},
    markers=True
)

# Customize the layout
fig.update_layout(
    xaxis_title='Month',
    yaxis_title='Average Sentiment Score',
    legend_title_text='Sentiment',
    xaxis=dict(tickmode='linear', showgrid=True, gridcolor='lightgray', gridwidth=0.5),  # Show all years on x-axis, add grid properties
    yaxis=dict(showgrid=True, gridcolor='lightgray', gridwidth=0.5), # Add grid properties to y-axis
)

# Show the plot
fig.show()